# Player Tracking System Comparison

This notebook compares 4 different player tracking systems:
- Eagle
- Darkmyter (using Ultralytics YOLO)
- AnshChoudhary
- TrackLab

**Important**: Run cells in order from top to bottom!

In [1]:
# Cell 1: Setup directories and utilities

from pathlib import Path
import os

BASE_DIR = Path("/content")
REPOS_DIR = BASE_DIR / "repositories"
VIDEOS_DIR = BASE_DIR / "videos"
CLIPS_DIR = BASE_DIR / "clips"
OUTPUT_DIR = BASE_DIR / "output"

for d in [REPOS_DIR, VIDEOS_DIR, CLIPS_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def print_status(msg, status="INFO"):
    """Print colored status messages"""
    colors = {
        "INFO": "\033[94m",
        "SUCCESS": "\033[92m",
        "WARNING": "\033[93m",
        "ERROR": "\033[91m",
        "RESET": "\033[0m"
    }
    print(f"{colors.get(status, '')}[{status}] {msg}{colors['RESET']}")

print_status("Directory structure created", "SUCCESS")
print(f"Working directory: {BASE_DIR}")

[SUCCESS] Directory structure created
Working directory: /content


In [2]:
# Cell 2: Clone all repositories

import subprocess

REPOSITORIES = {
    "eagle": "https://github.com/nreHieW/Eagle.git",
    "darkmyter": "https://github.com/Darkmyter/Football-Players-Tracking.git",
    "tracklab": "https://github.com/TrackingLaboratory/tracklab.git"
}

print_status("Cloning repositories...", "INFO")

for name, url in REPOSITORIES.items():
    repo_path = REPOS_DIR / name

    if repo_path.exists():
        print_status(f"{name}: Already exists, skipping", "WARNING")
        continue

    try:
        print_status(f"{name}: Cloning...", "INFO")
        result = subprocess.run(
            ["git", "clone", url, str(repo_path)],
            capture_output=True,
            text=True,
            timeout=300
        )

        if result.returncode == 0:
            print_status(f"{name}: Cloned successfully", "SUCCESS")
        else:
            print_status(f"{name}: Clone failed - {result.stderr[:100]}", "ERROR")

    except Exception as e:
        print_status(f"{name}: Clone failed - {str(e)}", "ERROR")

print_status("Repository cloning complete", "SUCCESS")

[INFO] Cloning repositories...
[INFO] eagle: Cloning...
[SUCCESS] eagle: Cloned successfully
[INFO] darkmyter: Cloning...
[SUCCESS] darkmyter: Cloned successfully
[INFO] tracklab: Cloning...
[SUCCESS] tracklab: Cloned successfully
[SUCCESS] Repository cloning complete


In [3]:
# Cell 3: Install dependencies

print_status("Installing dependencies...", "INFO")

!pip install -q torch torchvision torchaudio
!pip install -q opencv-python numpy scipy pandas scikit-learn matplotlib
!pip install -q ultralytics supervision
!pip install -q gdown Pillow tqdm requests

print_status("Dependencies installed", "SUCCESS")

[INFO] Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 17.8 MB/s eta 0:00:00
[SUCCESS] Dependencies installed


In [4]:
# Cell 4: Download videos from Google Drive

!pip install -q gdown

import gdown
from pathlib import Path

# Shared folder ID
FOLDER_ID = "1Cs4kTX6GYwfcpKyDZdqRKBezz49wT7_N"

print_status("Downloading videos from shared folder...", "INFO")

try:
    gdown.download_folder(
        id=FOLDER_ID,
        output=str(VIDEOS_DIR),
        quiet=False,
        use_cookies=False
    )

    # List downloaded videos
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.MP4', '.AVI', '.MOV', '.MKV']
    available_videos = []

    for ext in video_extensions:
        available_videos.extend(list(VIDEOS_DIR.glob(f"*{ext}")))

    if not available_videos:
        print_status("No video files found", "ERROR")
    else:
        print(f"DOWNLOADED {len(available_videos)} VIDEO(S)")


        for idx, video in enumerate(available_videos, 1):
            size_mb = video.stat().st_size / (1024 * 1024)
            print(f"{idx}. {video.name} ({size_mb:.1f} MB)")


        print("Enter video selection:")
        print("  - Leave blank to process ALL videos")
        print("  - Enter a number (e.g., '1')")
        print("  - Enter comma-separated numbers (e.g., '1,2')")

        selection = input("\nYour choice: ").strip()

        VIDEO_PATHS = []

        if not selection:
            VIDEO_PATHS = available_videos
            print_status(f"Selected ALL {len(VIDEO_PATHS)} videos", "SUCCESS")
        elif selection.isdigit():
            idx = int(selection)
            if 1 <= idx <= len(available_videos):
                VIDEO_PATHS = [available_videos[idx - 1]]
                print_status(f"Selected: {VIDEO_PATHS[0].name}", "SUCCESS")
        elif ',' in selection:
            try:
                indices = [int(x.strip()) for x in selection.split(',')]
                for idx in indices:
                    if 1 <= idx <= len(available_videos):
                        VIDEO_PATHS.append(available_videos[idx - 1])
                print_status(f"Selected {len(VIDEO_PATHS)} videos", "SUCCESS")
            except ValueError:
                print_status("Invalid input", "ERROR")

        if not VIDEO_PATHS:
            print_status("No videos selected", "ERROR")

except Exception as e:
    print_status(f"Download failed: {str(e)}", "ERROR")
    print("\nNote: Make sure the folder is set to 'Anyone with the link can view'")

[INFO] Downloading videos from shared folder...


Retrieving folder contents


Processing file 1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4
Processing file 1RvqkxASOD23jfigqSgSgGja5_NGZReO4 FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4
Processing file 1urwKF6wjitkREymiNi9O3jCLLIysTp6F FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf
From (redirected): https://drive.google.com/uc?id=1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf&confirm=t&uuid=032e891f-12f3-4e8e-b07b-acaba1d238f6
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4
100%|██████████| 1.68G/1.68G [00:16<00:00, 101MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1RvqkxASOD23jfigqSgSgGja5_NGZReO4
From (redirected): https://drive.google.com/uc?id=1RvqkxASOD23jfigqSgSgGja5_NGZReO4&confirm=t&uuid=7b5d111c-1633-4b08-81a4-8676299db9c2
To: /content/videos/FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4
100%|██████████| 1.92G/1.92G [00:19<00:00, 96.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1urwKF6wjitkREymiNi9O3jCLLIysTp6F
From (redirected): https://drive.google.com/uc?id=1urwKF6wjitkREymiNi9O3jCL

DOWNLOADED 3 VIDEO(S)
1. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4 (1604.1 MB)
2. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4 (1260.8 MB)
3. FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4 (1832.4 MB)
Enter video selection:
  - Leave blank to process ALL videos
  - Enter a number (e.g., '1')
  - Enter comma-separated numbers (e.g., '1,2')

Your choice: 3
[SUCCESS] Selected: FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4


In [5]:
# Cell 5: Extract clips

import cv2

CLIP_DURATION = 60
ALL_CLIPS = {}

for VIDEO_PATH in VIDEO_PATHS:
    VIDEO_NAME = VIDEO_PATH.stem


    print(f"PROCESSING: {VIDEO_NAME}\n")


    cap = cv2.VideoCapture(str(VIDEO_PATH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    cap.release()

    print(f"Duration: {duration:.1f}s | FPS: {fps:.1f} | Frames: {total_frames}")

    if duration < CLIP_DURATION * 3:
        if duration < CLIP_DURATION:
            CLIPS = [(0, duration, "full")]
        else:
            CLIPS = [(0, CLIP_DURATION, "start"), (max(duration - CLIP_DURATION, 0), CLIP_DURATION, "end")]
    else:
        CLIPS = [
            (0, CLIP_DURATION, "start"),
            ((duration - CLIP_DURATION) / 2, CLIP_DURATION, "middle"),
            (duration - CLIP_DURATION, CLIP_DURATION, "end")
        ]

    CLIP_PATHS = {}

    for start_time, clip_dur, position in CLIPS:
        clip_name = f"{VIDEO_NAME}_{position}.mp4"
        clip_path = CLIPS_DIR / clip_name

        cmd = ["ffmpeg", "-i", str(VIDEO_PATH), "-ss", str(start_time), "-t", str(clip_dur),
               "-c", "copy", str(clip_path), "-y", "-loglevel", "error"]

        result = subprocess.run(cmd, capture_output=True)

        if result.returncode == 0 and clip_path.exists():
            CLIP_PATHS[position] = clip_path
            print_status(f"Clip '{position}' extracted", "SUCCESS")

    ALL_CLIPS[VIDEO_NAME] = CLIP_PATHS

print(f"\nTotal: {sum(len(clips) for clips in ALL_CLIPS.values())} clips from {len(VIDEO_PATHS)} video(s)")

PROCESSING: FULL MATCH  Brazil v Mexico  World Cup 2018 720p

Duration: 6258.0s | FPS: 50.0 | Frames: 312900
[SUCCESS] Clip 'start' extracted
[SUCCESS] Clip 'middle' extracted
[SUCCESS] Clip 'end' extracted

Total: 3 clips from 1 video(s)


In [6]:
# Cell 6a: Setup Eagle properly

print_status("Setting up Eagle...", "INFO")

eagle_dir = REPOS_DIR / "eagle"

# Install Python 3.11 (PyTorch compatible)
print_status("Installing Python 3.11...", "INFO")
!apt-get update -qq
!apt-get install -y python3.11 python3.11-venv python3.11-dev
# Step 1: Install uv if not already installed
print_status("Installing uv...", "INFO")
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to PATH for this session
import os
os.environ['PATH'] = f"/root/.local/bin:{os.environ['PATH']}"

# Step 2: Download model weights
print_status("Downloading Eagle model weights...", "INFO")
models_dir = eagle_dir / "eagle" / "models"
if models_dir.exists():
    os.chdir(models_dir)
    !bash get_weights.sh
    os.chdir(BASE_DIR)
    print_status("Eagle weights downloaded", "SUCCESS")
else:
    print_status("Eagle models directory not found", "ERROR")

# Step 3: Create Eagle wrapper that uses their main.py with uv
eagle_wrapper = eagle_dir / "run_eagle.py"
eagle_wrapper.write_text('''
import argparse
import json
import subprocess
import sys
from pathlib import Path

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True)
    parser.add_argument("--output", required=True)
    parser.add_argument("--fps", default=24, type=int)
    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    print(f"Processing video: {video_path}")
    print(f"Output will be saved to: {output_path}")

    # Run Eagle using uv
    cmd = [
        "uv", "run", "main.py",
        "--video_path", str(video_path),
        "--fps", str(args.fps)
    ]

    print(f"Running command: {' '.join(cmd)}")

    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        cwd=Path(__file__).parent
    )

    print(result.stdout)

    # Check if the error is due to missing ball detection
    if result.returncode != 0:
        if "KeyError: 'Ball'" in result.stderr:
            print("Ball detection failed - extracting available player tracking data")
            # Try to extract whatever tracking data was generated before the ball error
            extract_available_data(video_path, output_path)
        else:
            print(result.stderr, file=sys.stderr)
            sys.exit(1)
    else:
        # Normal processing when Eagle succeeds
        extract_eagle_output(video_path, output_path)

def extract_eagle_output(video_path, output_path):
    """Extract Eagle's output data"""
    video_stem = video_path.stem
    eagle_output_dir = Path(__file__).parent / "output" / video_stem

    # Find the JSON output file (Eagle creates multiple files)
    json_files = list(eagle_output_dir.glob("*.json"))

    if not json_files:
        print("No JSON output found", file=sys.stderr)
        sys.exit(1)

    # Use the first JSON file or combine them
    tracking_file = eagle_output_dir / "tracking_data.json"
    if not tracking_file.exists():
        tracking_file = json_files[0]

    # Read Eagle's output and convert to our format
    with open(tracking_file, 'r') as f:
        eagle_data = json.load(f)

    # Convert Eagle format to our standard format
    standardized_output = {}

    # If Eagle output is already frame-indexed, use it directly
    if isinstance(eagle_data, dict):
        standardized_output = eagle_data

    # Save to our desired output location
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(standardized_output, f, indent=2)

    print(f"Eagle output saved: {len(standardized_output)} frames")

def extract_available_data(video_path, output_path):
    """Extract whatever tracking data Eagle generated before the ball error"""
    video_stem = video_path.stem
    eagle_output_dir = Path(__file__).parent / "output" / video_stem

    # Look for any tracking files that were created
    json_files = list(eagle_output_dir.glob("*.json"))
    csv_files = list(eagle_output_dir.glob("*.csv"))

    if json_files:
        # Use the most recent JSON file
        tracking_file = max(json_files, key=lambda x: x.stat().st_mtime)
        print(f"Found tracking data: {tracking_file}")

        with open(tracking_file, 'r') as f:
            tracking_data = json.load(f)

        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            json.dump(tracking_data, f, indent=2)

        print(f"Extracted available tracking data: {len(tracking_data)} frames")

    elif csv_files:
        # Try to convert CSV to our format
        import pandas as pd
        csv_file = max(csv_files, key=lambda x: x.stat().st_mtime)
        print(f"Found CSV data: {csv_file}")

        df = pd.read_csv(csv_file)
        tracking_data = {}

        # Convert DataFrame to our format
        for frame_idx in df['frame'].unique():
            frame_data = df[df['frame'] == frame_idx]
            players = []
            for _, row in frame_data.iterrows():
                players.append({
                    "id": int(row['id']),
                    "x": float(row['x']),
                    "y": float(row['y']),
                    "w": float(row['w']),
                    "h": float(row['h'])
                })
            tracking_data[str(frame_idx)] = players

        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            json.dump(tracking_data, f, indent=2)

        print(f"Converted CSV to tracking data: {len(tracking_data)} frames")

    else:
        print("No tracking data found - Eagle failed before generating any output")
        # Create empty output as last resort
        import cv2
        cap = cv2.VideoCapture(str(video_path))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()

        empty_output = {}
        for i in range(frame_count):
            empty_output[str(i)] = []

        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            json.dump(empty_output, f, indent=2)

        print(f"Created empty output with {frame_count} frames")

if __name__ == "__main__":
    main()
''')

print_status("Eagle setup complete", "SUCCESS")

[INFO] Setting up Eagle...
[INFO] Installing Python 3.11...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.11 libpython3.11-dev libpython3.11-minimal libpython3.11-stdlib
  python3.11-minimal
Suggested packages:
  binfmt-support
The following NEW packages will be installed:
  libpython3.11 libpython3.11-dev libpython3.11-minimal libpython3.11-stdlib
  python3.11 python3.11-dev python3.11-minimal python3.11-venv
0 upgraded, 8 newly installed, 0 to remove and 46 not upgraded.
Need to get 16.5 MB of archives.
After this operation, 58.4 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.11-minimal amd64 3.11.14-

In [ ]:
# Cell 6b: Setup Darkmyter (ByteTrack + YOLO)

print_status("Setting up Darkmyter tracking...", "INFO")

import os
import subprocess
from pathlib import Path

darkmyter_dir = REPOS_DIR / "darkmyter"

# -------------------------------------------------------------------
# 1) Ensure Darkmyter's football-specific YOLOv8 weights are present
#    Using your Google Drive link:
#    https://drive.google.com/file/d/12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx/view
# -------------------------------------------------------------------

weights_dir = darkmyter_dir / "yolov8-weights"
weights_dir.mkdir(parents=True, exist_ok=True)

custom_weights = weights_dir / "yolov8l-football-players.pt"
gdrive_id = "12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx"

def download_darkmyter_weights():
    print_status("Downloading Darkmyter football weights with gdown...", "INFO")
    try:
        try:
            import gdown
        except ImportError:
            # Install gdown if not available
            subprocess.run(
                ["pip", "install", "gdown"],
                check=True
            )
            import gdown

        url = f"https://drive.google.com/uc?id={gdrive_id}"
        gdown.download(url, str(custom_weights), quiet=False)
        print_status("Darkmyter weights downloaded", "SUCCESS")
    except Exception as e:
        print_status(f"Failed to download Darkmyter weights: {e}", "ERROR")


# If file exists, sanity-check the header to detect HTML
if custom_weights.exists():
    try:
        with open(custom_weights, "rb") as f:
            header = f.read(16)
        if header.startswith(b"<"):
            # Looks like an HTML page, not a .pt file
            print_status(
                "Existing Darkmyter weights look like HTML (probably a Drive warning page). "
                "Re-downloading...",
                "ERROR"
            )
            custom_weights.unlink(missing_ok=True)
            download_darkmyter_weights()
        else:
            print_status("Darkmyter weights already present and look valid", "SUCCESS")
    except Exception as e:
        print_status(f"Error checking Darkmyter weights, re-downloading: {e}", "ERROR")
        custom_weights.unlink(missing_ok=True)
        download_darkmyter_weights()
else:
    download_darkmyter_weights()

# -------------------------------------------------------------------
# 2) Create Darkmyter wrapper: YOLOv8 + ByteTrack, football weights
# -------------------------------------------------------------------

darkmyter_wrapper = darkmyter_dir / "run_darkmyter.py"
darkmyter_wrapper.write_text('''
import argparse
import json
from pathlib import Path

import cv2
from ultralytics import YOLO


def load_model(repo_root: Path):
    """
    Use Darkmyter's football-specific weights if available & loadable.
    Fallback to generic yolov8x.pt otherwise.
    """
    custom_weights = repo_root / "yolov8-weights" / "yolov8l-football-players.pt"

    if custom_weights.exists():
        try:
            print(f"Trying Darkmyter football weights: {custom_weights}")
            return YOLO(str(custom_weights))
        except Exception as e:
            print(f"Failed to load Darkmyter weights: {e}")
            print("Falling back to generic yolov8x.pt.")
            return YOLO("yolov8x.pt")
    else:
        print("Darkmyter football weights not found; using generic yolov8x.pt instead.")
        return YOLO("yolov8x.pt")


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True, help="Path to input video clip (mp4).")
    parser.add_argument("--output", required=True, help="Path to JSON output file.")
    parser.add_argument("--conf", type=float, default=0.3,
                        help="Confidence threshold (default: 0.3).")
    parser.add_argument("--iou", type=float, default=0.5,
                        help="IoU threshold for tracking (default: 0.5).")
    parser.add_argument("--save-video", action="store_true",
                        help="Also save an annotated tracking video next to the JSON.")

    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        raise FileNotFoundError(f"Input video not found: {video_path}")

    repo_root = Path(__file__).resolve().parent
    model = load_model(repo_root)

    print(f"Processing video with YOLOv8 + ByteTrack: {video_path}")
    print(f"JSON output will be saved to: {output_path}")

    # Run tracking with ByteTrack (Ultralytics built-in tracker)
    results = model.track(
        source=str(video_path),
        tracker="bytetrack.yaml",
        conf=args.conf,
        iou=args.iou,
        persist=True,
        verbose=False,
    )

    output_data = {}
    total_detections = 0

    # Optional annotated video output
    cap = None
    writer = None
    if args.save_video:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            print("WARNING: Could not open video for annotated output; continuing without --save-video.")
            cap = None
        else:
            fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            video_out_path = output_path.with_suffix(".mp4")
            video_out_path.parent.mkdir(parents=True, exist_ok=True)
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            writer = cv2.VideoWriter(str(video_out_path), fourcc, fps, (width, height))
            print(f"Annotated video will be saved to: {video_out_path}")

    for frame_idx, result in enumerate(results):
        tracks = []

        boxes = getattr(result, "boxes", None)
        if boxes is not None and boxes.id is not None:
            xywh = boxes.xywh.cpu().numpy()
            ids = boxes.id.cpu().numpy()
            confs = boxes.conf.cpu().numpy()

            for (x, y, w, h), track_id, conf in zip(xywh, ids, confs):
                tracks.append({
                    "id": int(track_id),
                    "x": float(x),
                    "y": float(y),
                    "w": float(w),
                    "h": float(h),
                    "confidence": float(conf),
                })

        output_data[str(frame_idx)] = tracks
        total_detections += len(tracks)

        # Write annotated frame if requested
        if writer is not None and cap is not None:
            ret, frame = cap.read()
            if not ret:
                break

            for t in tracks:
                x_c, y_c, w, h = t["x"], t["y"], t["w"], t["h"]
                x1 = int(x_c - w / 2)
                y1 = int(y_c - h / 2)
                x2 = int(x_c + w / 2)
                y2 = int(y_c + h / 2)

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    frame,
                    str(t["id"]),
                    (x1, max(0, y1 - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2,
                )

            writer.write(frame)

    if writer is not None:
        writer.release()
    if cap is not None:
        cap.release()

    # Save JSON output (the format your evaluation notebook expects)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open("w") as f:
        json.dump(output_data, f, indent=2)

    print(f"Processed {len(output_data)} frames with {total_detections} total detections")
    print(f"Output saved to: {output_path}")


if __name__ == "__main__":
    main()
''')

print_status("Darkmyter setup complete", "SUCCESS")


In [8]:
# ================================
# Cell: Complete TrackLab Setup (Local Editable Install + Fixes)
# ================================

print_status("Setting up TrackLab tracking system...", "INFO")

# --- Virtual display for any OpenCV / matplotlib GUI calls ---
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y xvfb > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
print_status("Virtual display started", "SUCCESS")

import os
import shutil
from pathlib import Path

tracklab_dir = REPOS_DIR / "tracklab"
venv_dir = tracklab_dir / ".venv"

# --- Clean slate: remove old .venv if it exists ---
if venv_dir.exists():
    print_status("Removing existing TrackLab .venv...", "INFO")
    shutil.rmtree(venv_dir)

# --- Create uv environment inside the repo ---
os.chdir(tracklab_dir)
print_status("Creating TrackLab uv environment...", "INFO")
!uv venv --python 3.12

# --- Install TrackLab from THIS repo, editable, with video extras ---
print_status("Installing TrackLab from local repo (editable, [video])...", "INFO")
# Note the -e .[video] instead of tracklab[video]
!uv pip install -q -e ".[video]"

# --- Smoke test the CLI ---
print_status("Testing TrackLab installation (tracklab --help)...", "INFO")
os.environ["MPLBACKEND"] = "Agg"
help_exit_code = os.system("uv run tracklab --help > /dev/null 2>&1")

if help_exit_code == 0:
    print_status("TrackLab base installation complete", "SUCCESS")
else:
    print_status("TrackLab help command failed - check logs above", "ERROR")

os.chdir(BASE_DIR)

# ====================
# APPLY ALL FIXES
# ====================

print_status("Applying fixes to TrackLab (local repo)...", "INFO")

# Resolve paths via tracklab_dir, not hard-coded /content paths
progress_path = tracklab_dir / "tracklab" / "callbacks" / "progress.py"
video_py_path = tracklab_dir / "tracklab" / "engine" / "video.py"

# --- Fix 1: Progress callback - handle missing 'name' field ---
print_status("Fix 1/4: Progress callback - missing 'name' field", "INFO")
!sed -i "s/video_metadata\\['name'\\]/video_metadata.get('name', 'unknown')/g" "{progress_path}"

# --- Fix 2: Progress callback - guard missing task progress bars ---
print_status("Fix 2/4: Progress callback - missing task bars", "INFO")
with open(progress_path, "r") as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    if "self.task_pbars[task].update()" in line and i > 0 and "if task in" not in lines[i - 1]:
        indent = len(line) - len(line.lstrip())
        lines[i] = (
            " " * indent + "if task in self.task_pbars:\n"
            + " " * (indent + 4) + "self.task_pbars[task].update()\n"
        )
        break

with open(progress_path, "w") as f:
    f.writelines(lines)

# --- Fix 3: Video engine - add image_pred parameter to callback ---
print_status("Fix 3/4: Video engine - add image_pred parameter", "INFO")
with open(video_py_path, "r") as f:
    content = f.read()

old_callback = """        detections = self.video_loop()
        self.callback(
            "on_video_loop_end",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            detections=detections,
        )"""

new_callback = """        detections = self.video_loop()
        self.callback(
            "on_video_loop_end",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            detections=detections,
            image_pred=pd.DataFrame(),
        )"""

if old_callback in content:
    content = content.replace(old_callback, new_callback)

# --- Fix 4: Video engine - wrap in tracker_state context manager ---
print_status("Fix 4/4: Video engine - add tracker_state context manager", "INFO")

old_code = """    def track_dataset(self):
        \"\"\"Run tracking on complete dataset.\"\"\"
        self.callback("on_dataset_track_start")
        self.callback(
            "on_video_loop_start",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            index=0,
        )
        detections = self.video_loop()
        self.callback(
            "on_video_loop_end",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            detections=detections,
            image_pred=pd.DataFrame(),
        )
        self.callback("on_dataset_track_end")
        return detections"""

new_code = """    def track_dataset(self):
        \"\"\"Run tracking on complete dataset.\"\"\"
        self.callback("on_dataset_track_start")

        # Use tracker_state context manager for proper save functionality
        video_id = 0
        with self.tracker_state(video_id):
            self.callback(
                "on_video_loop_start",
                video_metadata=pd.Series(name=self.video_filename),
                video_idx=0,
                index=0,
            )
            detections = self.video_loop()
            self.callback(
                "on_video_loop_end",
                video_metadata=pd.Series(name=self.video_filename),
                video_idx=0,
                detections=detections,
                image_pred=pd.DataFrame(),
            )

        self.callback("on_dataset_track_end")
        return detections"""

if old_code in content:
    content = content.replace(old_code, new_code)

with open(video_py_path, "w") as f:
    f.write(content)

print_status("All TrackLab fixes applied to local repo", "SUCCESS")

# --- Verify that patches landed in the right files ---
print_status("Verifying fixes...", "INFO")
!grep -q "video_metadata.get('name'" "{progress_path}" && echo "✓ Fix 1: Progress name field" || echo "✗ Fix 1 failed"
!grep -q "if task in self.task_pbars:" "{progress_path}" && echo "✓ Fix 2: Progress task bars" || echo "✗ Fix 2 failed"
!grep -q "image_pred=pd.DataFrame()" "{video_py_path}" && echo "✓ Fix 3: image_pred parameter" || echo "✗ Fix 3 failed"
!grep -q "with self.tracker_state(video_id):" "{video_py_path}" && echo "✓ Fix 4: Context manager" || echo "✗ Fix 4 failed"

print_status("TrackLab setup complete and ready for evaluation!", "SUCCESS")


[INFO] Setting up TrackLab tracking system...
[SUCCESS] Virtual display started
[INFO] Removing existing TrackLab .venv...
[INFO] Creating TrackLab uv environment...
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
[INFO] Installing TrackLab from local repo (editable, [video])...
[INFO] Testing TrackLab installation (tracklab --help)...
[SUCCESS] TrackLab base installation complete
[INFO] Applying fixes to TrackLab (local repo)...
[INFO] Fix 1/4: Progress callback - missing 'name' field
[INFO] Fix 2/4: Progress callback - missing task bars
[INFO] Fix 3/4: Video engine - add image_pred parameter
[INFO] Fix 4/4: Video engine - add tracker_state context manager
[SUCCESS] All TrackLab fixes applied to local repo
[INFO] Verifying fixes...
✓ Fix 1: Progress name field
✓ Fix 2: Progress task bars
✓ Fix 3: image_pred parameter
✗ Fix 4 failed
[SUCCESS] TrackLab setup complete and ready for evaluation!


In [12]:
# Cell: Create TrackLab wrapper (YOLOv8 + simple IoU tracker)

print_status("Creating TrackLab wrapper script (run_tracklab.py)...", "INFO")

from pathlib import Path
import textwrap

tracklab_dir = REPOS_DIR / "tracklab"
run_script = tracklab_dir / "run_tracklab.py"

run_script.write_text(textwrap.dedent(r'''
    #!/usr/bin/env python
    import argparse
    import json
    from pathlib import Path

    import cv2
    import numpy as np

    try:
        from ultralytics import YOLO
    except ImportError as e:
        raise SystemExit(
            "Ultralytics YOLO is not installed. "
            "Make sure tracklab[video] (or ultralytics) is installed in this environment."
        ) from e

    def iou(box1, box2):
        """Compute IoU between two [x1,y1,x2,y2] boxes."""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        inter_w = max(0.0, x2 - x1)
        inter_h = max(0.0, y2 - y1)
        inter = inter_w * inter_h

        if inter <= 0:
            return 0.0

        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - inter
        if union <= 0:
            return 0.0
        return float(inter / union)

    def main():
        parser = argparse.ArgumentParser()
        parser.add_argument("--video", required=True, help="Path to input video")
        parser.add_argument("--output", required=True, help="Path to JSON output")
        parser.add_argument("--conf_threshold", type=float, default=0.3)

        # These are accepted just so your SYSTEM_CONFIGS args don't crash
        parser.add_argument("--detector", default="yolov8")
        parser.add_argument("--tracker", default="simple_iou")

        args = parser.parse_args()

        video_path = Path(args.video)
        output_path = Path(args.output)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        if not video_path.exists():
            raise SystemExit(f"Input video not found: {video_path}")

        # Load YOLOv8 (COCO pretrained)
        # You can swap to another weights file if you want.
        model = YOLO("yolov8n.pt")

        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise SystemExit(f"Failed to open video: {video_path}")

        tracks = {}  # track_id -> last_bbox
        next_track_id = 1
        frame_idx = 0

        results = []

        while True:
            ok, frame = cap.read()
            if not ok:
                break

            # Run detection
            yolo_result = model(
                frame,
                conf=args.conf_threshold,
                verbose=False
            )[0]

            if yolo_result.boxes is None or len(yolo_result.boxes) == 0:
                frame_idx += 1
                continue

            boxes = yolo_result.boxes.xyxy.cpu().numpy()
            scores = yolo_result.boxes.conf.cpu().numpy()
            classes = yolo_result.boxes.cls.cpu().numpy().astype(int)

            # Very simple IoU-based multi-object tracker
            for box, score, cls_id in zip(boxes, scores, classes):
                # Optionally, restrict to people class (COCO class 0)
                # if cls_id != 0:
                #     continue

                best_tid = None
                best_iou = 0.0

                for tid, last_box in tracks.items():
                    i = iou(last_box, box)
                    if i > 0.5 and i > best_iou:
                        best_iou = i
                        best_tid = tid

                if best_tid is None:
                    best_tid = next_track_id
                    next_track_id += 1

                tracks[best_tid] = box

                results.append({
                    "frame_id": int(frame_idx),
                    "track_id": int(best_tid),
                    "bbox": [
                        float(box[0]),
                        float(box[1]),
                        float(box[2]),
                        float(box[3]),
                    ],
                    "score": float(score),
                    "class_id": int(cls_id),
                })

            frame_idx += 1

        cap.release()

        with output_path.open("w") as f:
            json.dump(results, f)

        print(f"Wrote {len(results)} detections to {output_path}")

    if __name__ == "__main__":
        main()
'''))

print_status("TrackLab wrapper script created at tracklab/run_tracklab.py", "SUCCESS")


[INFO] Creating TrackLab wrapper script (run_tracklab.py)...
[SUCCESS] TrackLab wrapper script created at tracklab/run_tracklab.py


In [13]:
import time
import json

"""
SYSTEM_CONFIGS = {
    "eagle": {
        "path": REPOS_DIR / "eagle",
        "script": "run_eagle.py"
    },

    "darkmyter": {
        "path": REPOS_DIR / "darkmyter",
        "script": "run_darkmyter.py"
    },
    "tracklab_yolov8_bytetrack": {
        "path": REPOS_DIR / "tracklab",
        "script": "run_tracklab.py",
        "args": ["--detector", "yolov8", "--tracker", "bytetrack", "--conf_threshold", "0.3"]
    },
}"""

# System configurations
SYSTEM_CONFIGS = {
    "tracklab_yolov8_bytetrack": {
        "path": REPOS_DIR / "tracklab",
        "script": "run_tracklab.py",
        "args": ["--detector", "yolov8", "--tracker", "bytetrack", "--conf_threshold", "0.3"]
    },
}

position_to_number = {"start": 1, "middle": 2, "end": 3, "full": 1}
def run_system_on_clip(system_name, system_config, video_name, clip_number, clip_path):
    """Run a tracking system on a clip"""

    output_dir = OUTPUT_DIR / video_name / "clips" / str(clip_number) / system_name
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / "output.json"

    print_status(f"Running {system_name} on {video_name}/clip_{clip_number}...", "INFO")

    # Change to system directory
    original_dir = os.getcwd()
    os.chdir(system_config["path"])

    # Build command
    if system_name == "eagle":
        cmd = [
            "uv", "run", "--python", "python3.11", "run_eagle.py",
            "--video", str(clip_path),
            "--output", str(output_file)
        ]
    elif system_name.startswith("tracklab"):
        # Use uv run to execute our wrapper *inside* the TrackLab env
        cmd = [
            "uv", "run", "python", system_config["script"],
            "--video", str(clip_path),
            "--output", str(output_file),
        ]

        # Pass through any extra args from SYSTEM_CONFIGS (detector/tracker/conf)
        for extra in system_config.get("args", []):
            cmd.append(str(extra))
    else:
        cmd = [
            "python", system_config["script"],
            "--video", str(clip_path),
            "--output", str(output_file)
        ]

    start_time = time.time()

    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=6000
        )

        elapsed = time.time() - start_time

        os.chdir(original_dir)

        # ADDED: Print stdout and stderr for debugging
        if result.stdout:
            print("=== STDOUT ===")
            print(result.stdout)
        if result.stderr:
            print("=== STDERR ===")
            print(result.stderr)

        if result.returncode == 0 and output_file.exists():
            try:
                with open(output_file) as f:
                    data = json.load(f)
                print_status(
                    f"{system_name} ({video_name}/clip_{clip_number}): SUCCESS in {elapsed:.1f}s",
                    "SUCCESS"
                )
                return {"success": True, "time": elapsed, "output": str(output_file), "frames": len(data)}
            except json.JSONDecodeError:
                print_status(
                    f"{system_name} ({video_name}/clip_{clip_number}): Invalid JSON output",
                    "ERROR"
                )
                return {"success": False, "time": elapsed, "error": "Invalid JSON"}
        else:
            error_msg = result.stderr if result.stderr else "Unknown error"
            print_status(
                f"{system_name} ({video_name}/clip_{clip_number}): FAILED - {error_msg[:200]}",  # Truncate long errors
                "ERROR"
            )
            return {"success": False, "time": elapsed, "error": error_msg}

    except subprocess.TimeoutExpired:
        os.chdir(original_dir)
        print_status(f"{system_name} ({video_name}/clip_{clip_number}): TIMEOUT", "ERROR")
        return {"success": False, "time": 600, "error": "Timeout"}

    except Exception as e:
        os.chdir(original_dir)
        print_status(f"{system_name} ({video_name}/clip_{clip_number}): EXCEPTION - {str(e)}", "ERROR")
        return {"success": False, "time": time.time() - start_time, "error": str(e)}
# Run all systems on all clips

print("STARTING EVALUATION\n")


all_results = {}

for video_name, clip_paths in ALL_CLIPS.items():

    print(f"VIDEO: {video_name}")


    video_results = {}

    for clip_position, clip_path in clip_paths.items():
        clip_number = position_to_number.get(clip_position, 1)

        print_status(f"Processing clip {clip_number} ({clip_position})...", "INFO")
        print("-" * 60)

        video_results[f"clip_{clip_number}"] = {}

        for system_name, system_config in SYSTEM_CONFIGS.items():
            result = run_system_on_clip(system_name, system_config, video_name, clip_number, clip_path)
            video_results[f"clip_{clip_number}"][system_name] = result

        print()

    all_results[video_name] = video_results

    # Save summary
    summary_file = OUTPUT_DIR / video_name / "summary.json"
    with open(summary_file, "w") as f:
        json.dump(video_results, f, indent=2)

    print_status(f"Summary saved: {summary_file}", "SUCCESS")

# Save overall summary
overall_summary = OUTPUT_DIR / "overall_summary.json"
with open(overall_summary, "w") as f:
    json.dump(all_results, f, indent=2)


print("EVALUATION COMPLETE\n")

print(f"\nResults: {OUTPUT_DIR}")

STARTING EVALUATION

VIDEO: FULL MATCH  Brazil v Mexico  World Cup 2018 720p
[INFO] Processing clip 1 (start)...
------------------------------------------------------------
[INFO] Running tracklab_yolov8_bytetrack on FULL MATCH  Brazil v Mexico  World Cup 2018 720p/clip_1...
=== STDOUT ===
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Wrote 5250 detections to /content/output/FULL MATCH  Brazil v Mexico  World Cup 2018 720p/clips/1/tracklab_yolov8_bytetrack/output.json

=== STDERR ===

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
100%|██████████| 6.25M/6.25M [00:00<00:00, 176MB/s]

[SUCCESS] tracklab_yolov8_bytetrack (FULL MATCH  Brazil v Mexico  World Cup 2018 720p/clip_1): SUCCESS in 46.1s

[INFO] Processing clip 2 (middle).

In [ ]:

import cv2
import json
import numpy as np

def create_overlay_video(video_path, json_path, output_path, show_ids=True, show_confidence=False):
    """
    Create an overlayed video with tracking results

    Args:
        video_path: Path to original video
        json_path: Path to tracking JSON output
        output_path: Path for output video
        show_ids: Whether to display player IDs
        show_confidence: Whether to display confidence scores
    """
    # Load tracking data
    with open(json_path, 'r') as f:
        tracking_data = json.load(f)

    # Open video
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    frame_idx = 0

    # Colors for different players (cycling through a palette)
    colors = [
        (0, 255, 0),    # Green
        (255, 0, 0),    # Blue
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 255, 0),  # Light Green
        (255, 128, 0),  # Light Blue
    ]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get tracking data for this frame
        frame_key = str(frame_idx)
        if frame_key in tracking_data:
            frame_data = tracking_data[frame_key]

            # Draw each player
            if 'Coordinates' in frame_data and 'Player' in frame_data['Coordinates']:
                players = frame_data['Coordinates']['Player']

                for player_id, player_data in players.items():
                    bbox = player_data['BBox']
                    confidence = player_data.get('Confidence', 0)

                    # Get color for this player ID
                    color = colors[int(player_id) % len(colors)]

                    # Draw bounding box
                    x1, y1, x2, y2 = bbox
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Prepare label
                    label_parts = []
                    if show_ids:
                        label_parts.append(f"ID:{player_id}")
                    if show_confidence:
                        label_parts.append(f"{confidence:.2f}")

                    if label_parts:
                        label = " ".join(label_parts)

                        # Draw label background
                        (label_width, label_height), _ = cv2.getTextSize(
                            label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                        )
                        cv2.rectangle(
                            frame,
                            (x1, y1 - label_height - 5),
                            (x1 + label_width, y1),
                            color,
                            -1
                        )

                        # Draw label text
                        cv2.putText(
                            frame,
                            label,
                            (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (255, 255, 255),
                            1
                        )

                    # Draw center point for transformed coordinates visualization
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    cv2.circle(frame, (center_x, center_y), 3, color, -1)

            # Add timestamp
            if 'Time' in frame_data:
                timestamp = frame_data['Time']
                cv2.putText(
                    frame,
                    f"Time: {timestamp}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2
                )

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()

    return True

print_status("Visualization function loaded", "SUCCESS")

In [ ]:
# Cell 8: Display results

import pandas as pd


print("RESULTS SUMMARY \n")


summary_data = []

for video_name, clips in all_results.items():
    for clip_key, systems in clips.items():
        for system_name, result in systems.items():
            summary_data.append({
                "Video": video_name,
                "Clip": clip_key,
                "System": system_name,
                "Status": "Valid" if result["success"] else "Invalid",
                "Time (s)": f"{result['time']:.1f}"
            })

df = pd.DataFrame(summary_data)
print(df.to_string(index=False))

total_runs = len(summary_data)
successful_runs = sum(1 for row in summary_data if row["Status"] == "Valid")


print(f"Success Rate: {successful_runs}/{total_runs} ({100*successful_runs/total_runs:.1f}%)\n")


In [ ]:

import cv2
import json
from pathlib import Path

def detect_json_format(data):
    """Detect which tracking system format the JSON is in"""
    if not data:
        return "unknown"

    first_frame_key = list(data.keys())[0]
    first_frame = data[first_frame_key]

    # Eagle format: nested dict with "Coordinates" -> "Player"
    if isinstance(first_frame, dict) and "Coordinates" in first_frame:
        if "Player" in first_frame["Coordinates"]:
            return "eagle"

    # AnshChoudhary/Darkmyter/TrackLab format: list of dicts with x, y, w, h
    if isinstance(first_frame, list) and len(first_frame) > 0:
        if "x" in first_frame[0] and "w" in first_frame[0]:
            return "xywh"

    return "unknown"

def parse_eagle_frame(frame_data):
    """Parse Eagle format frame data"""
    players = []
    if "Coordinates" in frame_data and "Player" in frame_data["Coordinates"]:
        for player_id, player_data in frame_data["Coordinates"]["Player"].items():
            bbox = player_data["BBox"]
            players.append({
                "id": int(player_id),
                "x1": bbox[0],
                "y1": bbox[1],
                "x2": bbox[2],
                "y2": bbox[3],
                "confidence": player_data.get("Confidence", 0)
            })
    return players, frame_data.get("Time", "")

def parse_xywh_frame(frame_data):
    """Parse AnshChoudhary/Darkmyter/TrackLab format frame data"""
    players = []
    if isinstance(frame_data, list):
        for player_data in frame_data:
            # Convert center x,y,w,h to corner coordinates
            center_x = player_data["x"]
            center_y = player_data["y"]
            w = player_data["w"]
            h = player_data["h"]

            x1 = int(center_x - w/2)
            y1 = int(center_y - h/2)
            x2 = int(center_x + w/2)
            y2 = int(center_y + h/2)

            players.append({
                "id": player_data["id"],
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
                "confidence": player_data.get("confidence", 0)
            })
    return players, ""

def create_universal_overlay_video(video_path, json_path, output_path, show_ids=True, show_confidence=False):
    """
    Create an overlayed video that works with ANY tracking system format
    """
    # Load tracking data
    with open(json_path, 'r') as f:
        tracking_data = json.load(f)

    # Detect format
    format_type = detect_json_format(tracking_data)
    print(f"    Detected format: {format_type}")

    if format_type == "unknown":
        print(f"    ✗ Unknown JSON format")
        return False

    # Open video
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    # Colors for different players
    colors = [
        (0, 255, 0),    # Green
        (255, 0, 0),    # Blue
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 255, 0),  # Light Green
        (255, 128, 0),  # Light Blue
        (0, 128, 255),  # Orange
        (255, 0, 128),  # Pink
    ]

    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get tracking data for this frame
        frame_key = str(frame_idx)
        if frame_key in tracking_data:
            frame_data = tracking_data[frame_key]

            # Parse based on format
            if format_type == "eagle":
                players, timestamp = parse_eagle_frame(frame_data)
            elif format_type == "xywh":
                players, timestamp = parse_xywh_frame(frame_data)
            else:
                players, timestamp = [], ""

            # Draw each player
            for player in players:
                player_id = player["id"]
                x1, y1, x2, y2 = player["x1"], player["y1"], player["x2"], player["y2"]
                confidence = player["confidence"]

                # Get color for this player ID
                color = colors[player_id % len(colors)]

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Prepare label
                label_parts = []
                if show_ids:
                    label_parts.append(f"ID:{player_id}")
                if show_confidence and confidence > 0:
                    label_parts.append(f"{confidence:.2f}")

                if label_parts:
                    label = " ".join(label_parts)

                    # Draw label background
                    (label_width, label_height), _ = cv2.getTextSize(
                        label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                    )
                    cv2.rectangle(
                        frame,
                        (x1, y1 - label_height - 5),
                        (x1 + label_width, y1),
                        color,
                        -1
                    )

                    # Draw label text
                    cv2.putText(
                        frame,
                        label,
                        (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255, 255, 255),
                        1
                    )

                # Draw center point
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                cv2.circle(frame, (center_x, center_y), 3, color, -1)

            # Add timestamp (for Eagle format)
            if timestamp:
                cv2.putText(
                    frame,
                    f"Time: {timestamp}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2
                )

            # Add frame number (for all formats)
            cv2.putText(
                frame,
                f"Frame: {frame_idx}",
                (10, height - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                1
            )

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()

    return True

# GENERATE OVERLAYS FOR ALL SYSTEMS

print("UNIVERSAL OVERLAY GENERATOR\n")

overlay_count = 0
failed_count = 0
system_counts = {}

for video_name, clips_data in all_results.items():
    print(f"\n📹 Video: {video_name}")

    for clip_key, systems_data in clips_data.items():
        # Reconstruct clip path
        clip_number = int(clip_key.split('_')[1])
        clip_path = None

        for pos, num in position_to_number.items():
            if num == clip_number and video_name in ALL_CLIPS and pos in ALL_CLIPS[video_name]:
                clip_path = ALL_CLIPS[video_name][pos]
                break

        if not clip_path:
            print(f" Skipping {clip_key} - clip path not found")
            continue

        print(f"\n {clip_key}:")

        for system_name, result_data in systems_data.items():
            if result_data.get('success'):
                try:
                    json_path = result_data['output']
                    output_dir = Path(json_path).parent
                    overlay_video_path = output_dir / "overlay_video.mp4"

                    print(f" {system_name}...", end=" ")

                    success = create_universal_overlay_video(
                        video_path=clip_path,
                        json_path=json_path,
                        output_path=overlay_video_path,
                        show_ids=True,
                        show_confidence=False
                    )

                    if success:
                        print(f"✓")
                        overlay_count += 1
                        system_counts[system_name] = system_counts.get(system_name, 0) + 1
                        result_data['overlay_video'] = str(overlay_video_path)
                    else:
                        print(f"✗")
                        failed_count += 1

                except Exception as e:
                    print(f"✗ Error: {e}")
                    failed_count += 1

print("SUMMARY\n")
print(f"Total overlays created: {overlay_count}")
print(f"Failed: {failed_count}")
print("\nPer System:")
for system, count in sorted(system_counts.items()):
    print(f"  • {system}: {count} video(s)")
print("="*60)

# Update summary files
for video_name, video_results in all_results.items():
    summary_file = OUTPUT_DIR / video_name / "summary.json"
    with open(summary_file, "w") as f:
        json.dump(video_results, f, indent=2)

with open(overall_summary, "w") as f:
    json.dump(all_results, f, indent=2)

print("\nSummary files updated with overlay paths!")

In [ ]:
# Cell 9: Download results

from google.colab import files
import shutil

print_status("Creating archive...", "INFO")

archive_name = "tracking_results"
archive_path = BASE_DIR / archive_name

shutil.make_archive(str(archive_path), 'zip', OUTPUT_DIR)

print_status("Downloading...", "SUCCESS")
files.download(f"{archive_path}.zip")

print_status("Complete!", "SUCCESS")